In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# === Step 1: Read and preprocess image ===
# For demonstration purposes, we'll use a built-in OpenCV image (like a checkerboard or synthetic shape)
# But here, we create a simple image with synthetic edges
#image = cv2.imread('circle.jpg', cv2.IMREAD_GRAYSCALE)
image = np.zeros((100, 100), dtype=np.uint8)
cv2.rectangle(image, (20, 20), (80, 80), 255, 2)  # draw a white rectangle

# The circle
#image_size = (100, 100)
#image = np.zeros(image_size, dtype=np.uint8)
#cv2.circle(image, center=(50, 50), radius=20, color=255, thickness=1)

In [ ]:
# === Step 2: Canny Edge Detection (custom implementation) ===
def custom_canny(img, low_threshold=200, high_threshold=210):
    from scipy.ndimage import gaussian_filter, sobel

    # Step 1: Noise Reduction
    blurred = gaussian_filter(img, sigma=1.4)

    # Step 2: Gradient Calculation (Sobel)
    gx = sobel(blurred, axis=1)
    gy = sobel(blurred, axis=0)
    grad = np.hypot(gx, gy)
    theta = np.arctan2(gy, gx)

    # Step 3: Non-Maximum Suppression
    nms = np.zeros_like(grad)
    angle = np.rad2deg(theta) % 180
    for i in range(1, img.shape[0] - 1):
        for j in range(1, img.shape[1] - 1):
            q, r = 255, 255
            a = angle[i, j]
            if (0 <= a < 22.5) or (157.5 <= a <= 180):
                q = grad[i, j + 1]
                r = grad[i, j - 1]
            elif 22.5 <= a < 67.5:
                q = grad[i + 1, j - 1]
                r = grad[i - 1, j + 1]
            elif 67.5 <= a < 112.5:
                q = grad[i + 1, j]
                r = grad[i - 1, j]
            elif 112.5 <= a < 157.5:
                q = grad[i - 1, j - 1]
                r = grad[i + 1, j + 1]
            if grad[i, j] >= q and grad[i, j] >= r:
                nms[i, j] = grad[i, j]

    # Step 4: Double Thresholding
    strong = 255
    weak = 75
    res = np.zeros_like(nms, dtype=np.uint8)
    strong_i, strong_j = np.where(nms >= high_threshold)
    weak_i, weak_j = np.where((nms <= high_threshold) & (nms >= low_threshold))
    res[strong_i, strong_j] = strong
    res[weak_i, weak_j] = weak

    # Step 5: Edge Tracking by Hysteresis
    for i in range(1, res.shape[0] - 1):
        for j in range(1, res.shape[1] - 1):
            if res[i, j] == weak:
                if ((res[i + 1, j - 1] == strong) or (res[i + 1, j] == strong) or (res[i + 1, j + 1] == strong)
                        or (res[i, j - 1] == strong) or (res[i, j + 1] == strong)
                        or (res[i - 1, j - 1] == strong) or (res[i - 1, j] == strong) or (res[i - 1, j + 1] == strong)):
                    res[i, j] = strong
                else:
                    res[i, j] = 0
    return res

edges = custom_canny(image)

In [ ]:
# === Step 3: Hough Transform (custom implementation) ===
def hough_transform(edge_img, rho_res=1, theta_res=1):
    height, width = edge_img.shape
    diag_len = int(np.ceil(np.sqrt(height**2 + width**2))) # Compute the max possible distance between a point and a line
    rhos = np.arange(-diag_len, diag_len + 1, rho_res) # Set up a list with these values
    thetas = np.deg2rad(np.arange(0, 180, theta_res)) # Generate a list of all possible angles
    accumulator = np.zeros((len(rhos), len(thetas)), dtype=np.uint64) # Build the accumulator array
    y_idxs, x_idxs = np.nonzero(edge_img) # Get the coordinates of the edges
    
    for i in range(len(x_idxs)): # solve p = y*sin(theta) + x*cos(theta) --> Get all possible lines the point could belong to
        x = x_idxs[i]
        y = y_idxs[i]
        for t_idx in range(len(thetas)):
            theta = thetas[t_idx]
            rho = int(round(x * np.cos(theta) + y * np.sin(theta)))
            accumulator[rho + diag_len, t_idx] += 1 # The voting step --> decides which line it actually belongs to --> all point which line they want to
            # belong two or more point choose the same line they ARE in the same line

    return accumulator, thetas, rhos

accumulator, thetas, rhos = hough_transform(edges)

In [ ]:
# === Step 4: Detect lines and draw them ===
threshold = 50
lines = np.argwhere(accumulator > threshold)
output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
diag_len = int(np.ceil(np.sqrt(image.shape[0]**2 + image.shape[1]**2)))

for rho_idx, theta_idx in lines:
    rho = rhos[rho_idx]
    theta = thetas[theta_idx]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a * rho
    y0 = b * rho
    x1 = int(x0 + 1000 * (-b))
    y1 = int(y0 + 1000 * (a))
    x2 = int(x0 - 1000 * (-b))
    y2 = int(y0 - 1000 * (a))
    cv2.line(output, (x1, y1), (x2, y2), (0, 0, 255), 1)

# === Step 5: Show results ===
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(image, cmap='gray')
axs[0].set_title("Original")
axs[1].imshow(edges, cmap='gray')
axs[1].set_title("Canny Edges")
axs[2].imshow(output)
axs[2].set_title("Hough Lines")
for ax in axs:
    ax.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# === Hough Circle Transform (custom implementation) ===
def hough_circle_transform(edge_img, radius_range):
    height, width = edge_img.shape
    min_r, max_r = radius_range
    accumulators = np.zeros((max_r - min_r + 1, height, width), dtype=np.uint64)
    y_idxs, x_idxs = np.nonzero(edge_img)

    for r_idx, r in enumerate(range(min_r, max_r + 1)):
        for i in range(len(x_idxs)):
            x = x_idxs[i]
            y = y_idxs[i]
            for theta in range(0, 360, 5):  # step size can be tuned
                t = np.deg2rad(theta)
                a = int(round(x - r * np.cos(t)))
                b = int(round(y - r * np.sin(t)))
                if 0 <= a < width and 0 <= b < height:
                    accumulators[r_idx, b, a] += 1
    return accumulators

# Run the Hough Circle Transform on the edge image
radius_range = (10, 30)
circle_accumulators = hough_circle_transform(edges, radius_range)

# === Detect circles ===
circle_candidates = []
threshold = 25  # votes threshold (tune as needed)
for r_idx in range(circle_accumulators.shape[0]):
    r = radius_range[0] + r_idx
    centers = np.argwhere(circle_accumulators[r_idx] > threshold)
    for y, x in centers:
        circle_candidates.append((x, y, r))

# === Draw detected circles on image ===
circle_output = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
for x, y, r in circle_candidates:
    cv2.circle(circle_output, (x, y), r, (0, 255, 0), 1)

# === Display results ===
fig, axs = plt.subplots(1, 3, figsize=(15, 5))
axs[0].imshow(image, cmap='gray')
axs[0].set_title("Original")
axs[1].imshow(edges, cmap='gray')
axs[1].set_title("Canny Edges")
axs[2].imshow(circle_output)
axs[2].set_title("Hough Circles")
for ax in axs:
    ax.axis('off')
plt.tight_layout()
plt.show()

In [2]:
import os
os.getcwd()

'C:\\Users\\samum\\forGithub'